# AgriMarket

## 농수산물


- Data Source : [농식품 빅데이터 거래소](https://kadx.co.kr/product/detail/94534d90-595d-11eb-acdd-9560621d9652)

- Collection Method : Download

- Data Format : CSV

### 데이터 설명

- 일반농산물 도매 69품목 116품종, 소매 90품목 143품종 및 친환경농산물 38품목 44종 대상 가격 자료


## 소비자 물가지수 


- Data Source : [국가통계포털](https://kosis.kr/statHtml/statHtml.do?orgId=101&tblId=DT_1J20112&conn_path=I2)

- Collection Method : Download

- Data Format : CSV

### 데이터 설명

- 농산물 중 필수재라고 판단되는 곡물,채소 포함

- 사치재인 과실, 기타 농산물 제외

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import koreanize_matplotlib
%config InlineBackend.figure_format = 'retina'

root_path = "../"
data_path = f"{root_path}data"

col_detail={
'prce_reg_ymd' : '가격등록일자',
'mrkt_esnt_no' : '시장고유번호',
'mrkt_code' : '시장코드',
'mrkt_nm' : '시장명',
'ct_code' : '시도코드',
'ctnp_nm' : '시도명',
'sggu_code' : '시군구코드',
'sggu_nm' : '시군구명',
'pdlt_code' : '품목코드',
'pdlt_nm' : '품목명',
'spcs_code' : '품종코드',
'spcs_nm' : '품종명',
'wsrt_exmn_se_code' : '도소매조사구분코드',
'exmn_se_nm' : '조사구분명',
'bulk_grad_code' : '산물등급코드',
'bulk_grad_nm' : '산물등급명',
'pdlt_prce' : '품목가격',
'mtc_smt_unit_mg' : '산지출하단위크기',
'mtc_smt_unit_nm' : '산지출하단위명',
'whsl_smt_unit_mg' : '도매출하단위크기',
'whsl_smt_unit_nm' : '도매출하단위명',
'rtsl_smt_unit_mg' : '소매출하단위크기',
'rtsl_smt_unit_nm' : '소매출하단위명',
'evfd_fmpd_smt_unit_mg' : '친환경농산물출하단위크기',
'evfd_fmpd_smt_unit_nm' : '친환경농산물출하단위명',
'dcnt_prce_yn' : '할인가격여부',
'etl_ldg_dt' : 'ETL적재일시'
}

# 파일 호출, 변수명 및 불러오기 경로는 자신에게 맞게 변경해주시면 됩니다.

In [2]:
# import os
# files_path = f'{data_path}/TB_TAT_KAMIS_PRC_MK_TB/'
# files = os.listdir(files_path)
# files.remove('.DS_Store')
# print(files)

In [3]:
# df = pd.DataFrame()
# for file in files:
#     df_sub = pd.read_csv(f'{files_path}/{file}', encoding='cp949', low_memory=False)
#     df_sub = df_sub[['PRCE_REG_YMD', 'CTNP_NM', 'PDLT_NM', 'EXMN_SE_NM', 'BULK_GRAD_NM', 'PDLT_PRCE']]
#     df_sub = df_sub[~df_sub['PDLT_NM'].isna()]
#     df_sub.columns=['YMD', 'Region', 'Product', 'WR', 'Grade', 'Price']
#     df = pd.concat([df, df_sub])

In [76]:
df = pd.read_csv(f"{data_path}/df_market.csv")
df = df.drop(columns="Region")
df

,YMD,Product,WR,Grade,Price,YM
0,20050103,쌀,도매,상품,40000.0,200501
1,20050103,쌀,도매,상품,40000.0,200501
2,20050103,쌀,도매,상품,40000.0,200501
3,20050103,쌀,도매,상품,40000.0,200501
4,20050103,쌀,도매,상품,40000.0,200501
...,...,...,...,...,...,...
14237905,20201231,청경채,07,무농약,4470.0,202012
14237906,20201231,팽이버섯,07,무농약,930.0,202012
14237907,20201231,상추,07,유기농,4940.0,202012
14237908,20201231,토마토,07,무농약,16800.0,202012


# 1. 년·월·일 에서 연·월 로 변경
# 2. Y(년), M(월) 컬럼 생성

In [77]:
df['YMD'] = df['YMD'].astype(str)

df['Y'] = df['YMD'].map(lambda x:x[:4]).astype('int16')
# df['M'] = df['YMD'].map(lambda x:x[4:6]).astype('int8')
# df['D'] = df['YMD'].map(lambda x:x[6:]).astype('int8')
# df['YM'] = df['YMD'].map(lambda x:x[:6]).astype('int32')

In [78]:
# 20년 데이터만 있는 품목 제외

df = df[~df['Product'].isin(['즉석밥', '두부', '김치', '고추장', '된장', '간장', '맛김(조미김)', '콩나물'])]

In [79]:
df.loc[df["Product"]=="피마늘", "Product"] = "마늘"

In [80]:
# Wholesale : 도매
df_w = df[df['WR']=='도매']

In [81]:
# Retail : 소매
df_r = df[df['WR']=='소매']

In [82]:
product_w = df_w['Product'].unique()
product_w = product_w.tolist()

In [83]:
product_r = df_r['Product'].unique()
product_r = product_r.tolist()

In [84]:
df_gw= pd.DataFrame(data=df_w.groupby(["YMD","Product"])["Price"].mean().reset_index())
df_gw['Y'] = df_gw['YMD'].map(lambda x:x[:4]).astype('int32')
df_gw['MD'] = df_gw['YMD'].map(lambda x:x[4:]).astype('str')

df_gw

,YMD,Product,Price,Y,MD
0,20050103,갈치,8900.000000,2005,0103
1,20050103,감귤,13466.000000,2005,0103
2,20050103,감자,21200.000000,2005,0103
3,20050103,건고추,461448.000000,2005,0103
4,20050103,건멸치,15200.000000,2005,0103
...,...,...,...,...,...
230781,20201231,팽이버섯,13130.000000,2020,1231
230782,20201231,포도,23790.000000,2020,1231
230783,20201231,풋고추,34323.333333,2020,1231
230784,20201231,피망,40720.000000,2020,1231


In [85]:
df_gr= pd.DataFrame(data=df_r.groupby(["YMD","Product"])["Price"].mean().reset_index())
df_gr['Y'] = df_gr['YMD'].map(lambda x:x[:4]).astype('int32')
df_gr['MD'] = df_gr['YMD'].map(lambda x:x[4:]).astype('str')
df_gr

,YMD,Product,Price,Y,MD
0,20050103,갈치,7360.606061,2005,0103
1,20050103,감귤,1793.461538,2005,0103
2,20050103,감자,250.897959,2005,0103
3,20050103,건고추,7034.864865,2005,0103
4,20050103,건멸치,2010.000000,2005,0103
...,...,...,...,...,...
250187,20201231,포도,32576.363636,2020,1231
250188,20201231,풋고추,932.195122,2020,1231
250189,20201231,피망,1073.191489,2020,1231
250190,20201231,호두,1466.341463,2020,1231


# 물가지수 파일과 겹치는 품목 확인

In [86]:
df_1 = pd.read_excel(f'{data_path}/소비자물가지수_품목별_연도별_월별.xlsx')
df_1

,시도별,품목별,2005.01,2005.02,2005.03,2005.04,2005.05,2005.06,2005.07,2005.08,...,2022.02,2022.03,2022.04,2022.05,2022.06,2022.07,2022.08,2022.09,2022.10,2022.11
0,전국,총지수,73.594,73.892,74.338,74.413,74.338,74.114,74.413,74.636,...,105.30,106.06,106.85,107.56,108.22,108.74,108.62,108.93,109.21,109.10
1,NaN,농축수산물,62.157,64.05,64.302,64.681,63.482,61.526,62.788,63.293,...,111.72,109.89,110.42,112.45,111.80,114.40,116.74,118.99,114.68,109.78
2,NaN,(농산물)곡물,76.091,76.015,75.866,75.641,76.015,76.015,75.866,75.716,...,103.59,102.59,101.13,99.72,98.51,97.32,95.47,93.93,94.40,97.81
3,NaN,쌀,75.464,75.539,75.391,75.093,75.242,75.316,75.168,75.019,...,102.87,101.90,99.94,98.23,96.77,95.23,92.80,90.34,91.08,96.16
4,NaN,현미,-,-,-,-,-,-,-,-,...,106.62,105.34,105.25,104.03,102.93,102.33,101.34,101.72,101.01,100.42
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58,NaN,(농산물)기타농산물,46.737,47.521,47.06,46.829,46.921,46.137,45.999,45.583,...,108.62,109.74,108.56,108.28,105.42,105.13,106.66,111.83,112.49,113.45
59,NaN,고춧가루,42.814,42.48,42.356,42.605,42.605,42.439,42.48,41.605,...,106.25,105.27,105.02,103.74,103.69,103.90,104.23,108.11,110.43,110.53
60,NaN,참깨,64.624,65.48,65.414,65.545,66.269,66.203,66.138,65.874,...,107.16,108.28,108.85,108.57,108.76,108.79,110.32,111.29,114.63,119.38
61,NaN,인삼,77.931,79.06,79.785,80.349,80.833,81.557,82.041,81.96,...,93.08,93.00,93.69,94.26,95.32,96.60,97.00,97.59,96.38,94.91


In [87]:
df_1['품목별'].unique()

array(['총지수', '농축수산물', '\u3000\u3000\u3000(농산물)곡물',
       '\u3000\u3000\u3000\u3000\u3000\u3000쌀',
       '\u3000\u3000\u3000\u3000\u3000\u3000현미',
       '\u3000\u3000\u3000\u3000\u3000\u3000찹쌀',
       '\u3000\u3000\u3000\u3000\u3000\u3000보리쌀',
       '\u3000\u3000\u3000\u3000\u3000\u3000콩',
       '\u3000\u3000\u3000\u3000\u3000\u3000땅콩',
       '\u3000\u3000\u3000\u3000\u3000\u3000혼식곡',
       '\u3000\u3000\u3000(농산물)채소',
       '\u3000\u3000\u3000\u3000\u3000\u3000배추',
       '\u3000\u3000\u3000\u3000\u3000\u3000상추',
       '\u3000\u3000\u3000\u3000\u3000\u3000시금치',
       '\u3000\u3000\u3000\u3000\u3000\u3000양배추',
       '\u3000\u3000\u3000\u3000\u3000\u3000미나리',
       '\u3000\u3000\u3000\u3000\u3000\u3000깻잎',
       '\u3000\u3000\u3000\u3000\u3000\u3000부추',
       '\u3000\u3000\u3000\u3000\u3000\u3000무',
       '\u3000\u3000\u3000\u3000\u3000\u3000열무',
       '\u3000\u3000\u3000\u3000\u3000\u3000당근',
       '\u3000\u3000\u3000\u3000\u3000\u3000감자',
       '\u3000\u3000\u3000\u

In [88]:
# replace로 \u3000 값이 사라지지 않는다.. 이유는 모르겠음
df_1['품목별'] = df_1['품목별'].replace('\u3000', '')

In [89]:
df_1['품목별'] = df_1['품목별'].map(lambda x:x.strip(('\u3000')))

In [90]:
df['Product'].unique()

array(['쌀', '찹쌀', '콩', '팥', '녹두', '메밀', '고구마', '감자', '배추', '양배추', '시금치',
       '상추', '수박', '오이', '호박', '토마토', '무', '당근', '건고추', '풋고추', '붉은고추',
       '마늘', '깐마늘(국산)', '깐마늘(수입)', '양파', '파', '생강', '미나리', '깻잎', '피망',
       '참깨', '들깨', '땅콩', '느타리버섯', '팽이버섯', '사과', '배', '포도', '감귤', '단감',
       '바나나', '참다래', '오렌지', '방울토마토', '고등어', '갈치', '명태', '물오징어', '건멸치',
       '북어', '건오징어', '김', '건미역', '굴', '수입조기', '돼지고기', '닭고기', '계란', '우유',
       '딸기', '참외', '복숭아', '쇠고기', '파프리카', '멜론', '새송이버섯', '얼갈이배추', '열무',
       '건포도', '건블루베리', '레몬', '체리', '아몬드', '자몽', '호두', '고춧가루', '굵은소금',
       '멸치액젓', '새우젓', '망고', '파인애플', '갓', '가지', '부추', '새우', '전복', '꽁치',
       '브로콜리', '청경채', '케일', '양송이버섯', '표고버섯', '귀리', '보리', '수수', '율무',
       '양상추', '혼합곡'], dtype=object)

# 농수산물 데이터와 주요경제지표 합치기

In [91]:
file = f"{data_path}/주요경제지표_20052021.xlsx"
df_mei = pd.read_excel(file, engine="openpyxl")
print(df_mei.shape)
display(df_mei.head(df_mei.shape[0]))

(17, 11)


/opt/anaconda3/lib/python3.9/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,연도,생산자물가지수(2010=100),소비자물가지수(2015=100),경상수지(백만US$),자본수지(백만US$),외환보유액(백만US$),수출액(백만US$),수입액(백만US$),환율(원/US$),실업률(%),콜금리(연%)
0,2021,110.64,108.050,88302.7,-154.3,463118.360,644400.370,615093.450,1144.42,3.7,0.61
1,2020,104.01,105.420,75275.7,-386.3,443098.120,512788.730,467548.715,1180.05,3.9,0.70
2,2019,104.48,104.850,59670.1,-169.3,408816.146,542232.610,503342.947,1157.80,3.8,1.59
3,2018,104.46,104.450,76408.5,316.7,403694.264,604859.657,535202.428,1118.10,3.8,1.52
4,2017,102.53,102.930,78460.0,-26.8,389266.683,573694.421,478478.296,1071.40,3.7,1.26
5,2016,99.11,100.970,98677.4,-46.2,371101.605,495425.934,406192.887,1208.50,3.7,1.34
6,2015,100.95,100.000,105939.6,-60.2,367961.865,526756.498,436498.973,1172.00,3.6,1.65
7,2014,105.17,99.298,84374.0,-8.9,363592.748,572664.602,525514.506,1099.20,3.5,2.34
8,2013,105.73,98.048,81148.2,-27.0,346459.569,559632.400,515585.500,1055.30,3.1,2.59
9,2012,107.45,96.789,50835.0,-41.7,326968.393,547869.800,519584.500,1071.10,3.2,3.08


In [92]:
df_mei = df_mei[df_mei["연도"] <= 2020]

In [93]:
df_mei = df_mei.rename(columns={"연도" : "Y"})
df_mei

,Y,생산자물가지수(2010=100),소비자물가지수(2015=100),경상수지(백만US$),자본수지(백만US$),외환보유액(백만US$),수출액(백만US$),수입액(백만US$),환율(원/US$),실업률(%),콜금리(연%)
1,2020,104.01,105.420,75275.7,-386.3,443098.120,512788.730,467548.715,1180.05,3.9,0.70
2,2019,104.48,104.850,59670.1,-169.3,408816.146,542232.610,503342.947,1157.80,3.8,1.59
3,2018,104.46,104.450,76408.5,316.7,403694.264,604859.657,535202.428,1118.10,3.8,1.52
4,2017,102.53,102.930,78460.0,-26.8,389266.683,573694.421,478478.296,1071.40,3.7,1.26
5,2016,99.11,100.970,98677.4,-46.2,371101.605,495425.934,406192.887,1208.50,3.7,1.34
6,2015,100.95,100.000,105939.6,-60.2,367961.865,526756.498,436498.973,1172.00,3.6,1.65
7,2014,105.17,99.298,84374.0,-8.9,363592.748,572664.602,525514.506,1099.20,3.5,2.34
8,2013,105.73,98.048,81148.2,-27.0,346459.569,559632.400,515585.500,1055.30,3.1,2.59
9,2012,107.45,96.789,50835.0,-41.7,326968.393,547869.800,519584.500,1071.10,3.2,3.08
10,2011,106.71,94.717,18655.8,-112.0,306402.488,555213.700,524413.100,1153.30,3.4,3.09


In [94]:
df_market = pd.merge(df, df_mei)
df_market

,YMD,Product,WR,Grade,Price,YM,Y,생산자물가지수(2010=100),소비자물가지수(2015=100),경상수지(백만US$),자본수지(백만US$),외환보유액(백만US$),수출액(백만US$),수입액(백만US$),환율(원/US$),실업률(%),콜금리(연%)
0,20050103,쌀,도매,상품,40000.0,200501,2005,86.88,78.444,12654.8,-0.6,210390.703,284418.70,261238.300,1013.00,3.7,3.33
1,20050103,쌀,도매,상품,40000.0,200501,2005,86.88,78.444,12654.8,-0.6,210390.703,284418.70,261238.300,1013.00,3.7,3.33
2,20050103,쌀,도매,상품,40000.0,200501,2005,86.88,78.444,12654.8,-0.6,210390.703,284418.70,261238.300,1013.00,3.7,3.33
3,20050103,쌀,도매,상품,40000.0,200501,2005,86.88,78.444,12654.8,-0.6,210390.703,284418.70,261238.300,1013.00,3.7,3.33
4,20050103,쌀,도매,상품,40000.0,200501,2005,86.88,78.444,12654.8,-0.6,210390.703,284418.70,261238.300,1013.00,3.7,3.33
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14237905,20201231,청경채,07,무농약,4470.0,202012,2020,104.01,105.420,75275.7,-386.3,443098.120,512788.73,467548.715,1180.05,3.9,0.70
14237906,20201231,팽이버섯,07,무농약,930.0,202012,2020,104.01,105.420,75275.7,-386.3,443098.120,512788.73,467548.715,1180.05,3.9,0.70
14237907,20201231,상추,07,유기농,4940.0,202012,2020,104.01,105.420,75275.7,-386.3,443098.120,512788.73,467548.715,1180.05,3.9,0.70
14237908,20201231,토마토,07,무농약,16800.0,202012,2020,104.01,105.420,75275.7,-386.3,443098.120,512788.73,467548.715,1180.05,3.9,0.70


In [95]:
import sys
sys.path.append("../Import")
from definitions import crop_list2
crop_list = crop_list2
# crop_list = [
#     "마늘",
#     "깻잎",
#     "감자",
#     "고구마"
# ]

len(crop_list)

4

In [96]:
df_market = df_market[df_market["Product"].isin(crop_list)]
df_market["YM"] = df_market["YM"].astype("str")
df_market

/var/folders/fc/jd4xh9c54w36j4ktc_spsz0m0000gn/T/ipykernel_17477/3247609776.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_market["YM"] = df_market["YM"].astype("str")


,YMD,Product,WR,Grade,Price,YM,Y,생산자물가지수(2010=100),소비자물가지수(2015=100),경상수지(백만US$),자본수지(백만US$),외환보유액(백만US$),수출액(백만US$),수입액(백만US$),환율(원/US$),실업률(%),콜금리(연%)
72,20050103,고구마,도매,상품,13000.0,200501,2005,86.88,78.444,12654.8,-0.6,210390.703,284418.70,261238.300,1013.00,3.7,3.33
73,20050103,고구마,도매,상품,13000.0,200501,2005,86.88,78.444,12654.8,-0.6,210390.703,284418.70,261238.300,1013.00,3.7,3.33
74,20050103,고구마,도매,상품,14000.0,200501,2005,86.88,78.444,12654.8,-0.6,210390.703,284418.70,261238.300,1013.00,3.7,3.33
75,20050103,고구마,도매,상품,12000.0,200501,2005,86.88,78.444,12654.8,-0.6,210390.703,284418.70,261238.300,1013.00,3.7,3.33
76,20050103,고구마,도매,상품,12000.0,200501,2005,86.88,78.444,12654.8,-0.6,210390.703,284418.70,261238.300,1013.00,3.7,3.33
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14237865,20201231,고구마,07,무농약,3000.0,202012,2020,104.01,105.420,75275.7,-386.3,443098.120,512788.73,467548.715,1180.05,3.9,0.70
14237866,20201231,감자,07,무농약,3300.0,202012,2020,104.01,105.420,75275.7,-386.3,443098.120,512788.73,467548.715,1180.05,3.9,0.70
14237883,20201231,고구마,07,유기농,9800.0,202012,2020,104.01,105.420,75275.7,-386.3,443098.120,512788.73,467548.715,1180.05,3.9,0.70
14237884,20201231,감자,07,무농약,8630.0,202012,2020,104.01,105.420,75275.7,-386.3,443098.120,512788.73,467548.715,1180.05,3.9,0.70


In [97]:
df_market["Product"].nunique()

4

In [98]:
set(crop_list) - set(df_market["Product"])

set()

In [99]:
file = f"{data_path}/생산자물가지수_품목별_연도별_월별.xlsx"
df2 = pd.read_excel(file, engine="openpyxl")
df2 = df2.rename(columns={'계정코드별':'Product'})
df2["Product"] = df2["Product"].map(lambda x : x.split()[0])
df2 = df2.replace("-", np.NaN)
df2 = df2.dropna(axis=0)
df2 = df2[df2["Product"].isin(crop_list)]

for i in df2.columns:
    df2.columns = df2.columns.map(lambda x: x.replace(".",""))
    
df2 = pd.DataFrame(df2).T
df2 = df2.rename(columns=df2.iloc[0])
df2 = df2.drop(index='Product')
df2 = pd.DataFrame(df2.stack())
df2.columns = ['item_PPI']
df2

item_PPI
200501 감자     71.15
       고구마    53.12
       마늘     49.58
200502 감자     81.05
       고구마    56.55
...             ...
202210 고구마    86.18
       마늘     149.1
202211 감자    150.93
       고구마     91.9
       마늘    148.84

[645 rows x 1 columns]

In [100]:
df3 = pd.read_excel(f"{data_path}/소비자물가지수_품목별_연도별_월별.xlsx")
df3 = df3.replace("-", np.NaN)
df3 = df3.rename(columns={'품목별':'Product'})
df3 = df3.drop(columns="시도별")
df3 = df3.dropna(axis=0)
df3['Product'] = df3['Product'].map(lambda x:x.strip(('\u3000')))
df3 = df3[df3["Product"].isin(crop_list)]

for i in df3.columns:
    df3.columns = df3.columns.map(lambda x: x.replace(".",""))
    
df3 = pd.DataFrame(df3).T
df3 = df3.rename(columns=df3.iloc[0])
df3 = df3.drop(index='Product')
df3 = pd.DataFrame(df3.stack())
df3.columns = ['item_CPI']
df3

item_CPI
200501 깻잎    53.348
       감자    69.162
       고구마   41.811
       마늘     58.27
200502 깻잎    56.294
...             ...
202210 마늘    144.46
202211 깻잎     101.3
       감자    118.13
       고구마    81.28
       마늘    142.63

[860 rows x 1 columns]

In [101]:
df_eco = pd.concat([df2,df3], axis=1).reset_index()
df_eco.columns = ["YM", "Product", "item_PPI", "item_CPI"]
df_eco["YM"].astype("str")
df_eco

,YM,Product,item_PPI,item_CPI
0,200501,감자,71.15,69.162
1,200501,고구마,53.12,41.811
2,200501,마늘,49.58,58.27
3,200502,감자,81.05,74.989
4,200502,고구마,56.55,43.18
...,...,...,...,...
855,202207,깻잎,NaN,111.87
856,202208,깻잎,NaN,123.33
857,202209,깻잎,NaN,149.95
858,202210,깻잎,NaN,123.21


In [102]:
df_market.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 760800 entries, 72 to 14237901
Data columns (total 17 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   YMD                760800 non-null  object 
 1   Product            760800 non-null  object 
 2   WR                 760800 non-null  object 
 3   Grade              760800 non-null  object 
 4   Price              760800 non-null  float64
 5   YM                 760800 non-null  object 
 6   Y                  760800 non-null  int16  
 7   생산자물가지수(2010=100)  760800 non-null  float64
 8   소비자물가지수(2015=100)  760800 non-null  float64
 9   경상수지(백만US$)        760800 non-null  float64
 10  자본수지(백만US$)        760800 non-null  float64
 11  외환보유액(백만US$)       760800 non-null  float64
 12  수출액(백만US$)         760800 non-null  float64
 13  수입액(백만US$)         760800 non-null  float64
 14  환율(원/US$)          760800 non-null  float64
 15  실업률(%)             760800 non-null  float64
 16 

In [127]:

df_eco["item_PPI"] = df_eco["item_PPI"].astype("float")
df_eco["item_CPI"] = df_eco["item_CPI"].astype("float")
df_eco.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 860 entries, 0 to 859
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   YM        860 non-null    object 
 1   Product   860 non-null    object 
 2   item_PPI  645 non-null    float64
 3   item_CPI  860 non-null    float64
dtypes: float64(2), object(2)
memory usage: 27.0+ KB


In [128]:
df = pd.merge(df_market, df_eco, on=["Product", "YM"])
df

,YMD,Product,WR,Grade,Price,YM,Y,생산자물가지수(2010=100),소비자물가지수(2015=100),경상수지(백만US$),자본수지(백만US$),외환보유액(백만US$),수출액(백만US$),수입액(백만US$),환율(원/US$),실업률(%),콜금리(연%),item_PPI,item_CPI
0,20050103,고구마,도매,상품,13000.0,200501,2005,86.88,78.444,12654.8,-0.6,210390.703,284418.70,261238.300,1013.00,3.7,3.33,53.12,41.811
1,20050103,고구마,도매,상품,13000.0,200501,2005,86.88,78.444,12654.8,-0.6,210390.703,284418.70,261238.300,1013.00,3.7,3.33,53.12,41.811
2,20050103,고구마,도매,상품,14000.0,200501,2005,86.88,78.444,12654.8,-0.6,210390.703,284418.70,261238.300,1013.00,3.7,3.33,53.12,41.811
3,20050103,고구마,도매,상품,12000.0,200501,2005,86.88,78.444,12654.8,-0.6,210390.703,284418.70,261238.300,1013.00,3.7,3.33,53.12,41.811
4,20050103,고구마,도매,상품,12000.0,200501,2005,86.88,78.444,12654.8,-0.6,210390.703,284418.70,261238.300,1013.00,3.7,3.33,53.12,41.811
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
760795,20201231,마늘,도매,중품,36700.0,202012,2020,104.01,105.420,75275.7,-386.3,443098.120,512788.73,467548.715,1180.05,3.9,0.70,102.04,118.170
760796,20201231,마늘,도매,상품,75000.0,202012,2020,104.01,105.420,75275.7,-386.3,443098.120,512788.73,467548.715,1180.05,3.9,0.70,102.04,118.170
760797,20201231,마늘,도매,중품,55000.0,202012,2020,104.01,105.420,75275.7,-386.3,443098.120,512788.73,467548.715,1180.05,3.9,0.70,102.04,118.170
760798,20201231,마늘,도매,상품,50000.0,202012,2020,104.01,105.420,75275.7,-386.3,443098.120,512788.73,467548.715,1180.05,3.9,0.70,102.04,118.170


In [129]:
df = df[(df["WR"]=="도매") | (df["WR"]=="소매")]
df

,YMD,Product,WR,Grade,Price,YM,Y,생산자물가지수(2010=100),소비자물가지수(2015=100),경상수지(백만US$),자본수지(백만US$),외환보유액(백만US$),수출액(백만US$),수입액(백만US$),환율(원/US$),실업률(%),콜금리(연%),item_PPI,item_CPI
0,20050103,고구마,도매,상품,13000.0,200501,2005,86.88,78.444,12654.8,-0.6,210390.703,284418.70,261238.300,1013.00,3.7,3.33,53.12,41.811
1,20050103,고구마,도매,상품,13000.0,200501,2005,86.88,78.444,12654.8,-0.6,210390.703,284418.70,261238.300,1013.00,3.7,3.33,53.12,41.811
2,20050103,고구마,도매,상품,14000.0,200501,2005,86.88,78.444,12654.8,-0.6,210390.703,284418.70,261238.300,1013.00,3.7,3.33,53.12,41.811
3,20050103,고구마,도매,상품,12000.0,200501,2005,86.88,78.444,12654.8,-0.6,210390.703,284418.70,261238.300,1013.00,3.7,3.33,53.12,41.811
4,20050103,고구마,도매,상품,12000.0,200501,2005,86.88,78.444,12654.8,-0.6,210390.703,284418.70,261238.300,1013.00,3.7,3.33,53.12,41.811
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
760795,20201231,마늘,도매,중품,36700.0,202012,2020,104.01,105.420,75275.7,-386.3,443098.120,512788.73,467548.715,1180.05,3.9,0.70,102.04,118.170
760796,20201231,마늘,도매,상품,75000.0,202012,2020,104.01,105.420,75275.7,-386.3,443098.120,512788.73,467548.715,1180.05,3.9,0.70,102.04,118.170
760797,20201231,마늘,도매,중품,55000.0,202012,2020,104.01,105.420,75275.7,-386.3,443098.120,512788.73,467548.715,1180.05,3.9,0.70,102.04,118.170
760798,20201231,마늘,도매,상품,50000.0,202012,2020,104.01,105.420,75275.7,-386.3,443098.120,512788.73,467548.715,1180.05,3.9,0.70,102.04,118.170


In [130]:
df_1 = df.groupby(["YMD","Product"])[['Price','환율(원/US$)','콜금리(연%)', 'item_PPI', 'item_CPI']].agg("mean").reset_index()
df_1

,YMD,Product,Price,환율(원/US$),콜금리(연%),item_PPI,item_CPI
0,20050103,감자,6323.101449,1013.00,3.33,71.15,69.162
1,20050103,고구마,4291.111111,1013.00,3.33,53.12,41.811
2,20050103,깻잎,4534.333333,1013.00,3.33,NaN,53.348
3,20050103,마늘,20785.714286,1013.00,3.33,49.58,58.270
4,20050104,감자,6233.314286,1013.00,3.33,71.15,69.162
...,...,...,...,...,...,...,...
15875,20201230,마늘,53533.333333,1180.05,0.70,102.04,118.170
15876,20201231,감자,5094.163934,1180.05,0.70,107.24,102.430
15877,20201231,고구마,10700.294118,1180.05,0.70,137.90,96.130
15878,20201231,깻잎,7598.166667,1180.05,0.70,NaN,87.660


In [132]:
df_1.to_csv("../data/AgriMarket.csv", index = False)